# Bank of Canada - Governor's Challenge - Inflation Forecasting Project @ UBC

## ******************************************** DATA CLEANING *********************************************

### 1) Loading in Packages

In [19]:
install.packages(c("dplyr","tidyr","readr","ggplot2","stats","tseries","lmtest","openxlsx","readxl"))

Warning message:
“package ‘stats’ is a base package, and should not be updated”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [21]:
library(dplyr)
library(tidyr)
library(readr)
library(ggplot2)
library(stats)
library(tseries)
library(lmtest)
library(openxlsx)
library(readxl)

### 2) Krishna's Data

**Pre-Cleaned Dataset - Original data files + data cleaning code (STATA) is in Github**

In [22]:
krishna_data <- read_excel("Krishna_Data.xlsx")

head(krishna_data)

Date,Time,GDP,UNRATE,Labour_Prod,CEER,AWE,GOC5Y,BOS,IE1,⋯,CPI,Output_Gap,FAM_IO,GSCPI,Overnight_Rate,Infl_Exp_1y,Infl_Exp_2y,Job_Vacancy_Rate,Unit_Labour_Cost,USA_Interest_Rate
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2000-01-01,1,1514676,7.300000,84.076,102.4333,NA,NA,NA,NA,⋯,94.36667,2.2,78.2,-0.4191127,NA,NA,NA,NA,71.995,5.676667
2000-04-01,2,1532503,6.700000,84.845,101.5333,NA,NA,NA,NA,⋯,94.80000,2.4,79.3,0.1406780,NA,NA,NA,NA,73.192,6.273333
2000-07-01,3,1548212,6.833333,85.627,101.9600,NA,NA,NA,NA,⋯,95.66667,2.5,80.1,0.0702347,NA,NA,NA,NA,73.379,6.520000
2000-10-01,4,1551274,6.433333,85.213,100.1900,NA,NA,NA,NA,⋯,96.70000,2.0,80.6,-0.6925838,NA,NA,NA,NA,73.935,6.473333
2001-01-01,5,1560006,7.533333,85.495,100.2567,656.5100,5.125781,NA,NA,⋯,96.93333,1.7,80.8,-0.7325969,5.456990,NA,NA,NA,74.300,5.593333
2001-04-01,6,1564912,7.066667,86.122,100.4533,653.9833,5.335238,NA,25,⋯,98.20000,1.3,80.6,-0.8786915,4.632617,NA,NA,NA,74.328,4.326667


**We will get:**

1) **GDP**: Final consumption expenditure (Seasonally adjusted at annual rates - Chained 2017 dollars) - https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610010401

2) **GSCPI**: Global Supply Chain Pressure Index - Federal Reserve Bank of NY - https://www.newyorkfed.org/research/gscpi.html.

3) **Labour Productivity**: Total Economy (Index: 2017=100) - Seasonally Adjusted - https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610020701

4) **CEER**: Canadian Effective Exchange Rate Index - Bank of Canada - https://www.bankofcanada.ca/rates/exchange/canadian-effective-exchange-rates/

5) **USA Fed Rate** - Policy Interest Rate set by the Federal Reserve Bank of USA - FRED Database - https://fred.stlouisfed.org/series/FEDFUNDS

6) **Overnight Rate** - Bank of Canada Target of the Overnight Rate (2001Q1) - https://www.bankofcanada.ca/valet/observations/INDV39079/csv + data scraped from BOC

7) **Output Gap** - BOC Economic Slack Measure (Monetary Policy Report) - https://www.bankofcanada.ca/rates/indicators/capacity-and-inflation-pressures/product-market-definitions/

8) **Time** - Index of Date in Quarters starting from 2000-Q1

In [23]:
krishna_data_cleaned <- krishna_data %>%
  select(Date, Time, GDP, GSCPI, Labour_Prod, CEER, USA_Interest_Rate, Overnight_Rate, Output_Gap)

head(krishna_data_cleaned)


Date,Time,GDP,GSCPI,Labour_Prod,CEER,USA_Interest_Rate,Overnight_Rate,Output_Gap
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2000-01-01,1,1514676,-0.4191127,84.076,102.4333,5.676667,NA,2.2
2000-04-01,2,1532503,0.1406780,84.845,101.5333,6.273333,NA,2.4
2000-07-01,3,1548212,0.0702347,85.627,101.9600,6.520000,NA,2.5
2000-10-01,4,1551274,-0.6925838,85.213,100.1900,6.473333,NA,2.0
2001-01-01,5,1560006,-0.7325969,85.495,100.2567,5.593333,5.456990,1.7
2001-04-01,6,1564912,-0.8786915,86.122,100.4533,4.326667,4.632617,1.3


### 3) Seans's Data

**Pre-Cleaned Dataset** - I need Original data files + data cleaning code + sources for Transparency

In [24]:
sean_data <- read_excel("Sean_Data.xlsx")

sean_data <- sean_data |>
  rename(`CPI-Core`   = "Measure of core inflation based on a factor model, CPI-common (year-over-year percent change)",
         `CPI-Median` = "Measure of core inflation based on a weighted median approach, CPI-median (year-over-year percent change) 2 3 6 7 8",
         `CPI-Trim`   = "Measure of core inflation based on a trimmed mean approach, CPI-trim (year-over-year percent change) 2 3 6 8 9")

head(sean_data)

Date,GDP: Final consumption expenditure (Seasonally adjusted at annual rates) (Chained 2017 dollars),"Canadian Dollars to U.S. Dollar Spot Exchange Rate, Canadian Dollars to One U.S. Dollar, Quarterly, Not Seasonally Adjusted, AVERAGED",Unemployment Rate (SA),INDINF_INFEXP_BOND_Q,BOCI,CPI-Core,CPI-Median,CPI-Trim,CPI,output
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Q1 1990,892824,1.1823,7.3,NA,276.8267,3.966667,4.500000,4.733333,NA,2.0
Q2 1990,882793,1.1707,7.6,NA,266.5933,3.966667,4.033333,4.300000,NA,1.3
Q3 1990,888741,1.1530,8.5,NA,283.4333,3.866667,3.800000,4.066667,NA,0.4
Q4 1990,889602,1.1612,9.5,NA,286.9733,3.966667,3.900000,3.933333,NA,-0.8
Q1 1991,875730,1.1561,10.5,NA,260.4267,4.100000,3.966667,3.933333,NA,-2.3
Q2 1991,890979,1.1493,10.5,NA,256.7400,3.733333,3.733333,3.433333,NA,-2.3


**We will get:**

1) **CPI-Core**: Measure of core inflation based on a factor model, CPI-common (year-over-year percent change) - Source?

2) **CPI-Median**: Measure of core inflation based on a weighted median approach, CPI-median (year-over-year percent change) - Source?

3) **CPI-Trim**: Measure of core inflation based on a trimmed mean approach, CPI-trim (year-over-year percent change) - Source?


In [25]:
sean_data_cleaned <- sean_data |>
  slice(41:n()) |> ##filter the data to only keep values from 2000 Q1 onwards
  select(Date, `CPI-Core`, `CPI-Median`, `CPI-Trim`)

head(sean_data_cleaned)

Date,CPI-Core,CPI-Median,CPI-Trim
<chr>,<dbl>,<dbl>,<dbl>
Q1 2000,1.666667,1.366667,1.566667
Q2 2000,1.566667,1.500000,1.633333
Q3 2000,1.833333,1.633333,1.833333
Q4 2000,1.966667,1.800000,2.000000
Q1 2001,2.300000,2.100000,2.200000
Q2 2001,2.500000,2.200000,2.400000


### 4) Sahaj's Data

**Pre-Cleaned Dataset - All datasets are directly from CANSIM + FRED - Data cleaning code is in Github**

**We will get:**

1) **Supply Slack**: Manufacturers’ total inventory-to-sales ratio - StatCan Table 16-10-0047-01 (vector v803313)

2) **Policy Uncertainty Index**: Canadian Economic Policy Uncertainty Index - FRED Dataset series CANEPUINDXM

### Combined Businesses-Consumer Expectation Index

3) **Consumer Inflation Expectations**: Mid-term inflation expectations (2014 onwards) - BOC https://www.bankofcanada.ca/publications/canadian-survey-of-consumer-expectations/canadian-survey-of-consumer-expectations-survey-data/

4) **Business Outlook Survey**: Index made from data about firms's inflation expectation in the next 12 months. Particularly, whether businesses thought inflation was going to be BELOW-1, ONE-to-TWO, TWO-to-THREE, and ABOVE-3 percent - (how were these weighted?) - https://www.bankofcanada.ca/publications/bos/business-outlook-survey-data/

5) **Breakeven Inflation Rate (BEIR)**: Market-based breakeven inflation rate – BOC (Excel STATIC_ATABLE_V122544_V122553-BEIR.xlsx - source?)

### Labour Market Index

6) **R8** - Labour Underutilization Rate: Broad unemployment measure (SA) – StatCan Table 14-10-0077-01 (vector v2440393)

7) **Unemployment by Duration**: Share unemployed 1–4 weeks, 27+ weeks, etc. (SA) – StatCan Table 14-10-0342-01 (vectors v1078667526, v1078667742, v1078668066)

8) **Average Hours Worked**: Total employees, average actual weekly hours worked (SA) – StatCan Table 14-10-0032-01 (vector v2685138)

9) **Average Hourly Wage**: Employees, average hourly wage rate (SA) – StatCan Table 14-10-0063-01 (vector v2132579)

10) **Prime-age Participation Rate**: Participation rate, ages 25–54 (SA) – StatCan LFS extract (Excel 1410028701-eng)

**These were combined to make:**

**Labour Market Index (LMI)**: Composite index built (using PCA) on unemployment, labour underutilization rate, long-term unemployment, job finding, separation, participation, hours worked, and wage growth 



